# Hugging Face Workshop

## Part 1 — What is Hugging Face?
(Explain what the Hub, models, datasets, and libraries are.)



## Part 2 — Downloading Datasets
(Code cells using `datasets` library.)



## Part 3 — Downloading Models with Transformers
(Code cells using `transformers` library.)



## Part 4 — Running LLMs Locally (Ollama & vLLM)
(Notes and optional code snippets showing usage.)